In [ ]:
import pandas as pd
import os

In [ ]:
# get the root directory, this notebook is in the notebooks directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
raw_files_dir = os.path.join(root_dir, 'data', 'raw')

In [ ]:
raw_files = os.listdir(raw_files_dir)
abs_raw_files = [os.path.join(raw_files_dir, f) for f in raw_files]

In [ ]:
transactions = pd.read_excel(abs_raw_files[0])

In [ ]:
transactions